# Contact Dynamics 

In this homework assignment, we will introduce contact dynamics! We will explore two different systems:

1. elastic impact using the double pendulum which you derived the dynamics to last week contacting a wall
2. ineleastic impact with a square rigid body contacting the floor

Visualization tools have already been provided to you for debugging. 

In [1]:
# Import packages
try:
    from jax import config
    config.update("jax_enable_x64", True)
    import meshcat
    import meshcat.geometry as geom
    import meshcat.transformations as tfm
    import numpy as onp
    import time
    import jax.numpy as np
    import jax
    from jax import jacfwd, hessian
    from jaxlie import SE2, SE3
    import matplotlib.pyplot as plt
    from viewer import DoublePendViewer, BlockRigidBodyViewer

    print('Import packages works! Great work following directions :D !')
except Exception as e:
    print('Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong')
    print(e)


# Constants for double pendulum
_l1 = 1.0
_l2 = 1.0
_m1 = 1.0
_m2 = 1.0
_g = 9.81

# constants for the 2D square
_m = 1.0
_a = 0.5
_I = (_a**4)/3.0 # moment of inertia for square
_M = onp.diag([_m, _m, _I])
_M_inv = np.linalg.inv(_M)

# Helper functions for integration
def rk4_step(f, x, dt):
    """
        Input:
            xdot = f(x) - function to be integrated, passed as f
            x - initial condition to the function
            dt - time step
        Output:
            x[t+dt]
    """
    # one step of runge-kutta integration
    k1 = f(x)
    k2 = f(x + dt*k1/2)
    k3 = f(x + dt*k2/2)
    k4 = f(x + dt*k3)
    xdot = x + 1/6*(k1+2*k2+2*k3+k4)*dt
    return xdot

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Import packages works! Great work following directions :D !


## Q1. Elastic Impact with the Double Pendulum 

We consider the double-pendulum from the previous homework assignment as our desired model. Fill in the code below and animate the contact! Below is a diagram of the double pendulum with a wall at the vertical y-axis (when x=0). 

<img src="./Double_Pendulum_With_Wall.png" width="420">


### Q1.a Fill in derived equations of motion from Lagrangian

Feel free to use previous homework assignments to fill in the code below. 

In [2]:
def p1(q):
    x1 = _l1 * np.sin(q[0])
    y1 = -_l1 * np.cos(q[0])
    return np.asarray([x1, y1])

def p2(q):
    x1 = _l1 * np.sin(q[0])
    y1 = -_l1 * np.cos(q[0])
    x2 =  x1 + _l2 * np.sin(q[0] + q[1])
    y2 =  y1 + -_l2 * np.cos(q[0] + q[1])
    return np.asarray([x2,y2])

jac_p1 = jacfwd(p1)
jac_p2 = jacfwd(p2)

def KE_derived(q, qdot):
    v1 = jac_p1(q)@qdot
    v2 = jac_p2(q)@qdot
    m1 = np.array([[_m1, 0], [0, _m1]])
    m2 = np.array([[_m2, 0], [0, _m2]])

    return 1/2 * v1.T @ m1 @ v1 + 1/2 * v2.T @ m1 @ v2

def PE_derived(q):
    _, y1 = p1(q)
    _, y2 = p2(q)
    return _m1 * _g * y1 + _m2 * _g * y2

def L_derived(q, qdot):
    return KE_derived(q, qdot) - PE_derived(q)

M_derived = jacfwd(jacfwd(L_derived, argnums=1), argnums=1)
C_derived = jacfwd(jacfwd(L_derived, argnums=1), argnums=0)
G_derived = jacfwd(L_derived)

def f_pendulum(x):
    q,qdot = np.split(x, 2)
    qddot = -np.linalg.inv(M_derived(q, qdot)) @ (C_derived(q, qdot)@ qdot - G_derived(q, qdot))
    xdot = np.array([qdot,qddot]).reshape((4,))
    return xdot
    return xdot

### Q1. b Contact condition
Create a function for the normal contact condition $\phi(q)$ for a vertical wall placed at $x=0$ in the world frame. Only consider contact with the center of mass at the second link. Also construct the contact normal Jacobian using `jacfwd`.

In [3]:
def phi(q):
    """
        Input: q
        Output: phi(q) distance to the contact wall
    """
    # FILL IN RESPONSE HERE
    return p2(q)[0]

contact_jac = jacfwd(phi)

Check your answer below. 

0.09983341664682815 [1.99500417 1.        ]

In [4]:
q_test = np.array([0.1,-0.1])
print(
    phi(q_test),
    contact_jac(q_test)
)

0.09983341664682815 [1.99500417 1.        ]


### Q1. c Elastic Impact Resolution 
In this question, you will construct the impact rule for what occurs after an impact is detected. 
Recall that the conditions for an *elastic* contact require moment to be conserved and that the transfer of moment be along the normal direction. We can summarize these conditions as follows

$$
\begin{align}
\frac{\partial L}{\partial \dot{q}} \Bigg\vert_{\tau^-}^{\tau+} &= J(q_\tau)^\top\lambda \\ 
\left[ \frac{\partial L}{\partial \dot{q}} \dot{q} - L(q, \dot{q})\right]_{\tau^-}^{\tau+} &= 0
\end{align}
$$

Where $J(q)$ is the contact Jacobian. Assuming a Lagrangian of the form $L(q,\dot{q}) = \frac{1}{2} \dot{q}^\top M(q) \dot{q} - V(q)$, we can rewrite the above condition as the following impact update rule. 

$$
\dot{q}_\tau^+ = \dot{q}_\tau^- + M^{-1} J^\top \lambda
$$
where the contact force is given by 

$$
\lambda = -2 \left( J M^{-1} J^\top \right)^{-1} J \dot{q}_\tau^-
$$

Assuming a coefficient of restitution of $1$. Write the following function using the update rule above. Recall that for this example, the contact only occurs at the last mass and is a scalar in the direction of the normal direction $\frac{\partial \phi}{\partial q}$, thus the inverse is a scalar inverse. 

In [5]:
def post_impact_update(q, qdot):
    """
        Input: q, qdot (minus)
        Output: qdot (plus post impact)
    """
    ### FILL IN EQUATIONS HERE
    lambda_val = -2 * 1/(contact_jac(q) @ np.linalg.inv(M_derived(q, qdot)) @ contact_jac(q).T) * contact_jac(q) @ qdot

    return qdot + np.linalg.inv(M_derived(q, qdot)) @ contact_jac(q).T * lambda_val

Check your answer below.

[ 1.1        -0.28900916]

In [6]:
q_test = np.array([0.1,-0.1])
qdot_test = np.array([1.1,-4.1])
print(
    post_impact_update(q_test, qdot_test)
)

[ 1.1        -0.28900916]


### Animate the solution with meshcat

In [7]:
viewer = DoublePendViewer()
viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [8]:
q = [np.pi/2, 0.]
qdot = [0., 0.]
x0 = np.array(q+qdot)
dt = 0.01

# joing to jit the functions here
f_jit = jax.jit(f_pendulum)
post_impact_update_jit = jax.jit(post_impact_update)

for t in range(1000):
    x0 = rk4_step(f_jit, x0, dt)
    _q, _qdot = np.split(x0,2)
    # for the elastic case, the update equations apply to qdot
    # so they can be applied externally
    if phi(_q) <= 0.:
        _qdot = post_impact_update_jit(_q,_qdot)
        x0 = np.hstack([_q,_qdot])
    time.sleep(dt)
    viewer.render(_q)


## Q2. inelastic contact with a 2D rigid body (a square)

In this example, we will consider the contact dynamics of a rigid body (a square) impacting the ground floor. Assume contact only occurs on the four corners of the square with side length $a$ (given above as a constant). To start, we will create the normal and tangent contact location functions for each possible contact. Here, the configuration of the box is $x, y, \theta$

<img src="./2D_Square_Contact_Dynamics.png" width="420">

### Q2.a Contact Jacobians

Here we will construct the normal and tangent contact Jacobians! We will model each contact at the corners of the square and check for contact at each iteration when stepping the dynamics. Fill in the functions below. 

In [9]:
k_d = 10.
k_n = 800.0
k_t = 600.
c_f = 10.

def phi_n(q):
    """
        Input: q
        Output: a list of all phi_n for all edges of the square with respect to the configuration of the square.
    """
    _x, _y, _th = q
    # FILL IN CODE HERE
    _dl = _a/np.sqrt(2.)
    _phi1 = _y - _dl * np.sin(np.pi/4 + _th)
    _phi2 = _y - _dl * np.sin(7 * np.pi/4 + _th)
    _phi3 = _y - _dl * np.sin(5 * np.pi/4 + _th)
    _phi4 = _y - _dl * np.sin(3 * np.pi/4 + _th)
    return [_phi1, _phi2, _phi3, _phi4]

def phi_t(q):
    """
        Input: q
        Output: a list of all phi_t contact location for all edges of the square with respect to the configuration of the square.
    """
    _x, _y, _th = q
    # FILL IN CODE HERE
    _dl = _a/np.sqrt(2.)
    _phi1 = _x - _dl * np.cos(np.pi/4 + _th)
    _phi2 = _x - _dl * np.cos(7 * np.pi/4 + _th)
    _phi3 = _x - _dl * np.cos(5 * np.pi/4 + _th)
    _phi4 = _x - _dl * np.cos(3 * np.pi/4 + _th)
    return [_phi1, _phi2, _phi3, _phi4]

# use the jacfwd function to get the contact jacobians
# in this case, the jacfwd function is applied to each element in the list above
J_n = jacfwd(phi_n)
J_t = jacfwd(phi_t)

# below is the contact jacobian for the ith contact point
# this function isolates the ith point from J_n and J_t
J_c = lambda i,q: np.vstack([J_n(q)[i], J_t(q)[i]])


Check your answer below. 

phi_n [Array(0.01640079, dtype=float64), Array(0.19757967, dtype=float64), Array(0.66359921, dtype=float64), Array(0.48242033, dtype=float64)]

 phi_t [Array(0.24242033, dtype=float64), Array(-0.22359921, dtype=float64), Array(-0.04242033, dtype=float64), Array(0.42359921, dtype=float64)] 
 J_n1 0.24272609986179147 
 J_t1 0.1970797630332925 
 J_n2 0.10292023696670753 
 J_t2 0.14272609986179147 
 J_n3 0.15727390013820855 
 J_t3 0.002920236966707516 
 J_n4 0.2970797630332925 
 J_t4 0.05727390013820856 

In [10]:
q_test = np.array([0.1, 0.34, 1.2])
qdot_test = np.array([0.1,0.2,0.3])
print(
    'phi_n' , phi_n(q_test), '\n',
    'phi_t' , phi_t(q_test),'\n',
    'J_n1' , J_n(q_test)[0]@qdot_test,'\n',
    'J_t1' , J_t(q_test)[0]@qdot_test,'\n',
    'J_n2' , J_n(q_test)[1]@qdot_test,'\n',
    'J_t2' , J_t(q_test)[1]@qdot_test,'\n',
    'J_n3' , J_n(q_test)[2]@qdot_test,'\n',
    'J_t3' , J_t(q_test)[2]@qdot_test,'\n',
    'J_n4' , J_n(q_test)[3]@qdot_test,'\n',
    'J_t4' , J_t(q_test)[3]@qdot_test,'\n',
)

phi_n [Array(0.01640079, dtype=float64), Array(0.19757967, dtype=float64), Array(0.66359921, dtype=float64), Array(0.48242033, dtype=float64)] 
 phi_t [Array(0.24242033, dtype=float64), Array(-0.22359921, dtype=float64), Array(-0.04242033, dtype=float64), Array(0.42359921, dtype=float64)] 
 J_n1 0.24272609986179147 
 J_t1 0.1970797630332925 
 J_n2 0.10292023696670752 
 J_t2 0.14272609986179147 
 J_n3 0.15727390013820855 
 J_t3 0.0029202369667075327 
 J_n4 0.2970797630332925 
 J_t4 0.05727390013820854 



### Q2.b Contact model

Here we will use a soft contact model to resolve the interaction with the square impacting the floor. Since we will not be using a strict elastic model, the contact forces will be **baked** into the dynamics. Use the notes to construct the contact model. Hint, you will use the `np.maximum` function and the `np.sign` or `np.linalg.norm` functions to calculate the contact forces.

In [11]:
def contact_model(_phi_n, v_n, v_t):
    """
        input:
            phi_n - normal penetration,
            v_n - normal velocity,
            v_t - tangent velocity
        output:
            lam - contact force np.array([lam_n, lam_t])
    """
    # FILL IN CODE HERE, note that lam_n and lam_t are scalars
    # lam_n = np.maximum(0, (-k_n * _phi_n - k_d * v_n))
    # lam_t = np.sign(v_t) * np.maximum(-k_t * lam_n, -c_f * abs(v_t))
    # return np.array([lam_n, lam_t])

Check your answer below. 

[1272.    5.]

In [12]:
_phi_n_test = -1.6
v_n_test = np.array(0.8)
v_t_test = np.array(-0.5)
print(
    contact_model(_phi_n_test, v_n_test, v_t_test)
)

[1272.    5.]


### Q2.c 2D rigid body dynamics for a square 

Use anyway to construct the dynamics of the rigid body. Include an if-statement for the contact conditions to be added into the equations of motion. 

NOTE: The simulation will be slow due to the many contact conditions. To speed things up we will use `jit` to compile singular contact functions as needed (you can't compile if-statements).

In [13]:
J_t = jax.jit(J_t)
J_n = jax.jit(J_n)
phi_n = jax.jit(phi_n)
phi_t = jax.jit(phi_t)
contact_model = jax.jit(contact_model)

def KE(q, qdot):
    v_x, v_y, v_th = qdot
    return (1/2) * _m * pow((np.sqrt(pow(v_x, 2) + pow(v_y, 2))), 2) + (1/2) * _I * (pow(v_th, 2))

def PE(q):
    x, y, th = q
    return _m * _g * y

def L(q, qdot):
    return KE(q, qdot) - PE(q)

C = jax.jit(jacfwd(jacfwd(L, argnums=1), argnums=0))
G = jax.jit(jacfwd(L, argnums=0))

def f_square(x):
    """
        Input:
            state = x = [q, qdot] -- the state of the system
        Output:
            xdot = [qdot, qddot]
    """
    q,qdot = np.split(x, 2)

    # FILL IN CODE HERE TO COMPUTE FREE DYNAMICS
    qddot = np.array([0, -_g, 0])

    # CHECK FOR CONTACTS HERE
    _phi_n_list = phi_n(q)
    # F_x collects all the contact forces in the generalized coordinates
    # i.e., sum J_c_i^\top lam_i
    F_x = 0.0
    for i, _phi_n in enumerate(_phi_n_list):
        if _phi_n < 0.0:
            # FILL IN CODE HERE TO COMPUTE CONTACT DYNAMICS

            # add in the contact Jacobian for one active contact at a time
            F_x = F_x + J_c(i, q).T @ contact_model(_phi_n, J_n(q)[i]@qdot, J_t(q)[i]@qdot)

    if np.any(F_x != 0.0):
        qddot = _M_inv @ (-C(q, qdot) @ qdot + G(q, qdot) + F_x)

    xdot = np.hstack([qdot, qddot])
    return xdot

In [1]:
viewer = BlockRigidBodyViewer()
viewer.jupyter_cell()

NameError: name 'BlockRigidBodyViewer' is not defined

### Animate the square

In [15]:
tf = 4
dt = 0.01
N = int(tf/dt)
q = [0., 2.0, 2.2]
qdot = [4., 0., 0.]
x0 = np.array(q + qdot)
for k in range(N):
    x0 = rk4_step(f_square, x0, dt)
    q,qdot = np.split(x0, 2)
    viewer.render(q)
    time.sleep(dt)